In [2]:
import numpy as np

result = np.zeros((10,3))
for i in range(1,11):
    result[i-1] = np.array([i,i,i])

result

array([[ 1.,  1.,  1.],
       [ 2.,  2.,  2.],
       [ 3.,  3.,  3.],
       [ 4.,  4.,  4.],
       [ 5.,  5.,  5.],
       [ 6.,  6.,  6.],
       [ 7.,  7.,  7.],
       [ 8.,  8.,  8.],
       [ 9.,  9.,  9.],
       [10., 10., 10.]])

In [3]:
np.percentile(result[:,0],78, interpolation='nearest')

8.0

In [4]:
from lidar_compression.util.pointcloud_sort import remove_distant_and_close_points

points = np.zeros((10,3))
for i in range(1,11):
    points[i-1] = np.array([i,i,i])

correct_result = np.zeros((6,3))
for i in range(3,9):
    correct_result[i-3] = np.array([i,i,i])


reduced_points = remove_distant_and_close_points(points, min_percentile=22, max_percentile=78)
print(reduced_points)
print(correct_result)

[[3. 3. 3.]
 [4. 4. 4.]
 [5. 5. 5.]
 [6. 6. 6.]
 [7. 7. 7.]
 [8. 8. 8.]]
[[3. 3. 3.]
 [4. 4. 4.]
 [5. 5. 5.]
 [6. 6. 6.]
 [7. 7. 7.]
 [8. 8. 8.]]


In [5]:
import numpy as np
import unittest
import tensorflow as tf
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

from lidar_compression.util.pointcloud_sort import add_spherical_coordinates, remove_distant_and_close_points, sort_points
from lidar_compression.util.pointcloud_sort_optimizations import sort_points_theta_phi_naive

dir = "/disk/vanishing_data/fa401/waymo/"
filename = "segment-10023947602400723454_1120_000_1140_000_with_camera_labels.tfrecord"
file = dir + filename
dataset = tf.data.TFRecordDataset(file)
for data in dataset:
    frame = open_dataset.Frame()
    frame.ParseFromString(bytearray(data.numpy()))
    break
(range_images, camera_projections,
    range_image_top_pose) = frame_utils.parse_range_image_and_camera_projection(frame)
points, _ = frame_utils.convert_range_image_to_point_cloud(
    frame,
    range_images,
    camera_projections,
    range_image_top_pose)
# 3d points in vehicle frame.
cloud_3d = np.concatenate(points, axis=0)

cartesian_spherical_cloud = add_spherical_coordinates(cloud_3d)
sorted_cloud = sort_points(cartesian_spherical_cloud)
sorted_cloud_naive = sort_points_theta_phi_naive(cartesian_spherical_cloud)

2021-08-19 19:57:03.171419: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-19 19:57:04.953569: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-19 19:57:04.969955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-19 19:57:04.970355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2021-08-19 19:57:04.970371: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-19 19:57:04.971947: I tensorflow/stream_execut

In [6]:
cartesian_points = np.array([[1,2,3], [-1,-2,-3], [5,6,3], [-5,3,10], [0,0,0], [10,10,10]], dtype=np.float)
cartesian_spherical_cloud = add_spherical_coordinates(cartesian_points)
sorted_cloud = sort_points(cartesian_spherical_cloud, 6,6)
sorted_cloud_naive = sort_points_theta_phi_naive(cartesian_spherical_cloud,6,6)

In [7]:
print(sorted_cloud)
print(sorted_cloud_naive)

print(np.array_equal(sorted_cloud, sorted_cloud_naive))

[[[ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 1.   2.   3. ]
  [ 0.   0.   0. ]
  [-5.   3.  10. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 7.5  8.   6.5]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [-1.  -2.  -3. ]]]
[[[ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 1.   2.   3. ]
  [ 0.   0.   0. ]
  [-5.   3.  10. ]
  [ 0.   0.   0. ]
  [ 0.   0.   0. ]]

 [[ 0.   0.   0. ]
  [ 7.5  8.   6.5]
  [ 0.   0.   0. ]
  [ 0.   0.   0

In [8]:
correct_result_xy = np.zeros((6,2))
correct_result_z = np.zeros((6,1))
for i in range(3,9):
    correct_result_xy[i-3] = np.array([i,i])
for i in range(9,2,-1):
    correct_result_z[8-i] = np.array([i])
correct_result = np.concatenate([correct_result_xy, correct_result_z], axis=1)

correct_result

array([[3., 3., 8.],
       [4., 4., 7.],
       [5., 5., 6.],
       [6., 6., 5.],
       [7., 7., 4.],
       [8., 8., 3.]])

In [9]:
result_xy = np.zeros((10,2))
result_z = np.zeros((10,1))
for i in range(1,11):
    result_xy[i-1] = np.array([i,i])
for i in range(10,0,-1):
    result_z[10-i] = np.array([i])
result = np.concatenate([result_xy, result_z], axis=1)

result

array([[ 1.,  1., 10.],
       [ 2.,  2.,  9.],
       [ 3.,  3.,  8.],
       [ 4.,  4.,  7.],
       [ 5.,  5.,  6.],
       [ 6.,  6.,  5.],
       [ 7.,  7.,  4.],
       [ 8.,  8.,  3.],
       [ 9.,  9.,  2.],
       [10., 10.,  1.]])

In [12]:
result[2:8]

array([[3., 3., 8.],
       [4., 4., 7.],
       [5., 5., 6.],
       [6., 6., 5.],
       [7., 7., 4.],
       [8., 8., 3.]])

In [11]:
np.percentile(result, 100)

10.0